In [1]:
!gcloud auth activate-service-account --key-file=noted-branch-140913-9bde9d49c946.json

'gcloud' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
%pip install google-cloud-language
%pip install pypdf2

   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.4 MB 3.4 MB/s eta 0:00:02
   ----------- ---------------------------- 1.3/4.4 MB 3.4 MB/s eta 0:00:01
   ------------------- -------------------- 2.1/4.4 MB 3.3 MB/s eta 0:00:01
   -------------------------- ------------- 2.9/4.4 MB 3.6 MB/s eta 0:00:01
   -------------------------------------- - 4.2/4.4 MB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 4.4/4.4 MB 4.1 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/user/Developer/Code/AI/nlp_google/noted-branch-140913-9bde9d49c946.json"


In [4]:
import PyPDF2

def pdf_to_text(pdf_path):
    # Membuka file PDF
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        
        # Inisialisasi variabel untuk menyimpan teks
        text = ""
        
        # Loop melalui setiap halaman dan ekstrak teks
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()  # Ekstrak teks dari halaman
        
    return text

# Contoh penggunaan
pdf_path = "/Users/user/Developer/Code/AI/nlp_google/cv.pdf"
text = pdf_to_text(pdf_path)

# Simpan hasil ke file teks
with open("output.txt", "w", encoding="utf-8") as output_file:
    output_file.write(text)

print("Konversi selesai!")



FileNotFoundError: [Errno 2] No such file or directory: '/Users/user/Developer/Code/AI/nlp_google/cv.pdf'

In [7]:
from google.cloud import language_v1

def analyze_sentiment(text):
    client = language_v1.LanguageServiceClient()

    document = language_v1.Document(
        content=text,
        type_=language_v1.Document.Type.PLAIN_TEXT
    )

    response = client.analyze_sentiment(request={'document': document})
    sentiment = response.document_sentiment

    print(f"Score Sentiment: {sentiment.score}")
    print(f"Magnitudo Sentiment: {sentiment.magnitude}")

text = "I hate you !!!"
analyze_sentiment(text)

Score Sentiment: -0.8999999761581421
Magnitudo Sentiment: 0.8999999761581421


In [11]:
from google.cloud import language_v1
import re

# Function to analyze entities
def analyze_entities(text):
    client = language_v1.LanguageServiceClient()
    document = language_v1.Document(
        content=text,
        type_=language_v1.Document.Type.PLAIN_TEXT,
        language="en"  # Setting the language to English for entity extraction
    )

    response = client.analyze_entities(request={'document': document})
    entities = {}

    # Process the entities
    for entity in response.entities:
        # If the entity is a person, we'll store it as a full name
        if language_v1.Entity.Type(entity.type_).name == 'PERSON':
            entities['full_name'] = entity.name

    # Manually extract additional entities using regular expressions and keywords

    # Extract NIDN (16-digit ID)
    nidn_match = re.search(r'\d{16}', text)  # NIDN usually has 16 digits
    if nidn_match:
        entities['nidn'] = nidn_match.group()

    # Extract Education (S1, S2, or degree related terms)
    education_keywords = ["S1", "S2", "Bachelor", "Master", "Doctor"]
    education = [word for word in text.split() if word in education_keywords]
    if education:
        entities['education'] = " ".join(education)

    # Extract Work History (Position, Job)
    work_keywords = ["position", "job", "Lecturer", "Assistant Expert", "IT Staff"]
    work_history = [word for word in text.split() if word.lower() in work_keywords]
    if work_history:
        entities['work_history'] = " ".join(work_history)

    # Extract Research History (Research, Thesis, Project)
    research_keywords = ["research", "study", "project", "Thesis", "researcher"]
    research_history = [word for word in text.split() if word.lower() in research_keywords]
    if research_history:
        entities['research_history'] = " ".join(research_history)

    # Extract Skills (Programming languages, frameworks, etc.)
    skills_keywords = ["Python", "Java", "Machine Learning", "AI", "CodeIgniter", "Deep Learning"]
    skills = [word for word in text.split() if word in skills_keywords]
    if skills:
        entities['skills'] = " ".join(skills)

    return entities

# Function to read and analyze file content
def read_and_analyze_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    entities = analyze_entities(text)
    return entities

# Example usage
file_path = '/Users/user/Developer/Code/AI/nlp_google/output.txt'  # Path to your converted text file
entities = read_and_analyze_file(file_path)

print("Analysis Results:")
for key, value in entities.items():
    print(f"{key}: {value}")

Analysis Results:
full_name: Sarman Chisara
nidn: 1989071120240610
education: S1 S2
work_history: Job Job Position Position
research_history: Project Research Research
skills: CodeIgniter AI


In [16]:
%pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 2.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
import fitz  # PyMuPDF
from google.cloud import language_v1
import re

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Function to analyze entities
def analyze_entities(text):
    client = language_v1.LanguageServiceClient()
    document = language_v1.Document(
        content=text,
        type_=language_v1.Document.Type.PLAIN_TEXT,
        language="en"  # Setting the language to English for entity extraction
    )

    response = client.analyze_entities(request={'document': document})
    entities = {}

    # Process the entities
    for entity in response.entities:
        # If the entity is a person, we'll store it as a full name
        if language_v1.Entity.Type(entity.type_).name == 'PERSON':
            entities['full_name'] = entity.name

    # Manually extract additional entities using regular expressions and keywords

    # Extract NIDN (16-digit ID)
    nidn_match = re.search(r'\d{16}', text)  # NIDN usually has 16 digits
    if nidn_match:
        entities['nidn'] = nidn_match.group()

    # Extract Education (S1, S2, or degree related terms)
    education_keywords = ["S1", "S2", "Bachelor", "Master", "Doctor"]
    education = [word for word in text.split() if word in education_keywords]
    if education:
        entities['education'] = " ".join(education)

    # Extract Work History (Position, Job)
    work_keywords = ["position", "job", "Lecturer", "Assistant Expert", "IT Staff"]
    work_history = [word for word in text.split() if word.lower() in work_keywords]
    if work_history:
        entities['work_history'] = " ".join(work_history)

    # Extract Research History (Research, Thesis, Project)
    research_keywords = ["research", "study", "project", "Thesis", "researcher"]
    research_history = [word for word in text.split() if word.lower() in research_keywords]
    if research_history:
        entities['research_history'] = " ".join(research_history)

    # Extract Skills (Programming languages, frameworks, etc.)
    skills_keywords = ["Python", "Java", "Machine Learning", "AI", "CodeIgniter", "Deep Learning"]
    skills = [word for word in text.split() if word in skills_keywords]
    if skills:
        entities['skills'] = " ".join(skills)

    return entities

# Function to read and analyze a PDF file
def analyze_pdf_file(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    entities = analyze_entities(text)
    return entities

# Example usage
pdf_path = '/Users/user/Developer/Code/AI/nlp_google/cv.pdf'  # Path to your PDF file
entities = analyze_pdf_file(pdf_path)

print("Analysis Results:")
for key, value in entities.items():
    print(f"{key}: {value}")


Analysis Results:
full_name: Tim Sumber Anggaran
nidn: 1989071120240610
education: S1 S1 S2
work_history: Position
skills: AI


In [19]:
# Contoh sederhana untuk analisis entitas dari teks menggunakan Google NLP API
simple_text = "John Doe has a Bachelor degree in Computer Science and works as a Lecturer. He has skills in Python and AI."

# Menggunakan fungsi analyze_entities untuk menganalisis teks sederhana
simple_entities = analyze_entities(simple_text)

print("Hasil Analisis Entitas:")
for key, value in simple_entities.items():
    print(f"{key}: {value}")

Hasil Analisis Entitas:
full_name: Lecturer
education: Bachelor
skills: Python


In [20]:
def analyze_entities(text):
    client = language_v1.LanguageServiceClient()
    document = language_v1.Document(
        content=text,
        type_=language_v1.Document.Type.PLAIN_TEXT
    )

    response = client.analyze_entities(request={'document': document})
    for entity in response.entities:
        print(f"Nama Entitas: {entity.name}")
        print(f"Tipe Entitas: {language_v1.Entity.Type(entity.type_).name}")
        print(f"Salience: {entity.salience}")

text = "Google is a leading tech company based in Mountain View, California."
analyze_entities(text)

Nama Entitas: Google
Tipe Entitas: ORGANIZATION
Salience: 0.8855599761009216
Nama Entitas: California
Tipe Entitas: LOCATION
Salience: 0.07895361632108688
Nama Entitas: Mountain View
Tipe Entitas: LOCATION
Salience: 0.035486411303281784
